## Проект по SQL

### Описание проекта
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. В нашем распоряжении имеется база данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цель проекта: проанализировать базу данных.

### Описание данных

Таблица books содержит данные о книгах:
- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- num_pages — количество страниц
- publication_date — дата публикации книги
- publisher_id — идентификатор издателя

Таблица authors содержит данные об авторах:
- author_id — идентификатор автора
- author — имя автора

Таблица publishers содержит данные об издательствах:
- publisher_id — идентификатор издательства
- publisher — название издательства

Таблица ratings содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

Таблица reviews содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора
- book_id — идентификатор книги
- username — имя автора обзора
- text — текст обзора

### План выполнения задания
- Описание исследования
- Исследование таблиц — вывод первых строк
- Сделать по одному SQL-запросу для решения каждого задания
- Вывести результаты каждого запроса в тетрадке
- Описать выводы по каждой из решённых задач

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


## Исследование таблиц

### Таблица books содержит данные о книгах

In [3]:
query = '''
SELECT COUNT(*)
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors содержит данные об авторах:

In [5]:
query = '''
SELECT COUNT(*)
FROM authors
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


In [6]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers содержит данные об издательствах:

In [7]:
query = '''
SELECT COUNT(*)
FROM publishers
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,340


In [8]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings содержит данные о пользовательских оценках книг:


In [9]:
query = '''
SELECT COUNT(*)
FROM ratings
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,6456


In [10]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews содержит данные о пользовательских обзорах:


In [11]:
query = '''
SELECT COUNT(*)
FROM reviews
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


In [12]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Вывод
- В таблице books содержится информация о 1000 книгах
- В таблице authors содержится информация о 636 авторах
- В таблице publishers содержится информация о 340 издательствах
- В таблице ratings содержится информация о 6456 оценках
- В таблице reviews содержится информация о 2793 пользовательских обзорах

## SQL-запросы для решения каждого задания

### Посчитаю, сколько книг вышло после 1 января 2000 года

In [13]:
query = '''
SELECT COUNT(DISTINCT book_id) as kolichestvo_knig
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,kolichestvo_knig
0,819


### Вывод
- После 1 января вышло 819 книг

### Для каждой книги посчитаю количество обзоров и среднюю оценку

In [14]:
query = '''
SELECT title as nazvanie,
       COUNT(DISTINCT review_id) as kolichestvo_obzorov,
       AVG(rating) as sredni_rating
FROM books as b
LEFT JOIN reviews as r ON b.book_id = r.book_id
FULL JOIN ratings as re ON b.book_id = re.book_id
GROUP BY nazvanie
ORDER BY kolichestvo_obzorov DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,nazvanie,kolichestvo_obzorov,sredni_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


### Вывод
- Максимальное количество обзоров (8) и рейтинг (4.138) имеет книга Мемуары Гейши

### Определю издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим  из анализа брошюры

In [15]:
query = '''
SELECT publisher as izdatelstvo,
       COUNT(book_id) as kolichestvo_knig
FROM books as b
INNER JOIN publishers as pbs ON b.publisher_id = pbs.publisher_id
WHERE num_pages > 50
GROUP BY izdatelstvo
ORDER BY kolichestvo_knig DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,izdatelstvo,kolichestvo_knig
0,Penguin Books,42


### Вывод 
- Издательство, которое выпустило наибольшее число книг(42) толще 50 страниц именуется Penguin Books

### Определю автора с самой высокой средней оценкой книг — учитываю только книги с 50 и более оценками

ЭТО ДЕЛАЛ ДЛЯ СЕБЯ, ЧТОБЫ ТАК СКАЗАТЬ, НЕ ДЕРЖАТЬ ЧАСТЬ КОДА В ГОЛОВЕ:
Тут решил сделать декомпозицию и разбить задачу на несколько частей. Сначала вывел таблицу с айди книги, авторами и названиями книг

In [16]:
query = '''
SELECT author,
       book_id,
       title           
FROM books as b
LEFT JOIN authors as atr ON b.author_id = atr.author_id
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,book_id,title
0,Stephen King/Jerry N. Uelsmann,1,'Salem's Lot
1,Patricia Schultz,2,1 000 Places to See Before You Die
2,Maureen Johnson,3,13 Little Blue Envelopes (Little Blue Envelope...
3,Charles C. Mann,4,1491: New Revelations of the Americas Before C...
4,David McCullough,5,1776
...,...,...,...
995,Terry Pratchett,996,Wyrd Sisters (Discworld #6; Witches #2)
996,Orson Scott Card/Piotr W. Cholewa,997,Xenocide (Ender's Saga #3)
997,Geraldine Brooks,998,Year of Wonders
998,Christopher Moore,999,You Suck (A Love Story #2)


Затем вывел таблицу с айди книги, количеством оценок и средним рейтингом

In [17]:
query = '''
SELECT DISTINCT b.book_id,
       COUNT(rating) as kolichestvo_ocenok,
       AVG(rating) as srednaya_ocenka
FROM books as b
LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
GROUP BY b.book_id
ORDER BY b.book_id ASC

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,kolichestvo_ocenok,srednaya_ocenka
0,1,3,3.666667
1,2,2,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,6,4.000000
...,...,...,...
995,996,3,3.666667
996,997,5,3.400000
997,998,5,3.200000
998,999,2,4.500000


С помощью табличного выражения получил нужный результат

In [18]:
query = '''
WITH
x AS (SELECT book_id,
             title,
             author       
      FROM books as b
      LEFT JOIN authors as atr ON b.author_id = atr.author_id),

y AS (SELECT DISTINCT b.book_id,
             COUNT(rating) as kolichestvo_ocenok,
             AVG(rating) as srednaya_ocenka
      FROM books as b
      LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
      GROUP BY b.book_id
      ORDER BY COUNT(rating) DESC)

SELECT x.author,
       AVG(y.srednaya_ocenka) AS srednaya_ocenka
FROM x 
LEFT OUTER JOIN y ON x.book_id = y.book_id
WHERE kolichestvo_ocenok > 50
GROUP BY x.author
ORDER BY srednaya_ocenka DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,srednaya_ocenka
0,J.K. Rowling/Mary GrandPré,4.283844


### Вывод 
Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré, а средняя оценка 4.283844

### Посчитаю среднее количество обзоров от пользователей, которые поставили больше 48 оценок

ЭТО ДЕЛАЛ ДЛЯ СЕБЯ, ЧТОБЫ ТАК СКАЗАТЬ, НЕ ДЕРЖАТЬ ЧАСТЬ КОДА В ГОЛОВЕ:
Так же использовал декомпозицию. Сначала нашёл пользователей и количество обзоров

In [19]:
query = '''
SELECT DISTINCT username,
       COUNT(text) as kolichestvo_obzorov
FROM books as b
LEFT JOIN reviews as rvw ON b.book_id = rvw.book_id
GROUP BY username
ORDER BY kolichestvo_obzorov DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,username,kolichestvo_obzorov
0,susan85,29
1,sfitzgerald,28
2,martinadam,27
3,eallen,26
4,lesliegibbs,26
...,...,...
156,joseph14,10
157,todd65,10
158,robert00,9
159,millernicholas,5


Затем нашёл пользователей и количество оценок

In [20]:
query = '''
SELECT username,
       COUNT(rating) as kolichestvo_ocenok
FROM books as b
LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
GROUP BY username
ORDER BY COUNT(rating) DESC

'''
pd.io.sql.read_sql(sql=text(query), con = con)

,username,kolichestvo_ocenok
0,martinadam,56
1,paul88,56
2,sfitzgerald,55
3,richard89,55
4,jennifermiller,53
...,...,...
155,lunakathryn,30
156,conradanita,30
157,wsingleton,30
158,robert00,29


С помощью табличного выражения получил нужный результат

In [21]:
query = '''
WITH
x AS (SELECT DISTINCT username,
             COUNT(text) as kolichestvo_obzorov
      FROM books as b
      LEFT JOIN reviews as rvw ON b.book_id = rvw.book_id
      GROUP BY username
      ORDER BY kolichestvo_obzorov DESC),

y AS (SELECT username,
             COUNT(rating) as kolichestvo_ocenok
      FROM books as b
      LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
      GROUP BY username
      ORDER BY COUNT(rating) DESC)

SELECT AVG(x.kolichestvo_obzorov) as srednee
FROM x 
LEFT OUTER JOIN y ON x.username = y.username
WHERE kolichestvo_ocenok > 48
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,srednee
0,24.0


### Вывод 
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок около 24